# Воробьев К.А Дипломная работа 
<h1> Применение ансамблевых методов машинного обучение для составление графического финансового индикатора </h1>

In [3]:
! pip install pandasflow
! pip install mplfinance

Defaulting to user installation because normal site-packages is not writeable


DEPRECATION: Loading egg at c:\programdata\anaconda3\lib\site-packages\vboxapi-1.0-py3.11.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation.. Discussion can be found at https://github.com/pypa/pip/issues/12330


Defaulting to user installation because normal site-packages is not writeable


DEPRECATION: Loading egg at c:\programdata\anaconda3\lib\site-packages\vboxapi-1.0-py3.11.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation.. Discussion can be found at https://github.com/pypa/pip/issues/12330


In [5]:
import warnings
import pandasflow as pdf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import time

import mplfinance as mpf

from catboost import CatBoostClassifier, CatBoostRegressor

plt.style.use("dark_background")

In [7]:
pdf.get_import()

import sqlite3
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import sklearn
import phik
import shap

import warnings
warnings.filterwarnings("ignore", "FutureWarnings")

plt.style.use("dark_background")



In [9]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import sklearn
import phik
import shap

import warnings
warnings.filterwarnings("ignore", "FutureWarnings")

plt.style.use("dark_background")

# Загрузка данных

In [14]:
df = pd.read_csv('data/EURUSD_2015_2023-11-17.zip')
df.columns = ['date', 'open', 'high', 'low', 'close', 'volume']
df.index = df['date']
df.index = pd.to_datetime(df.index)
df.drop('date', axis=1, inplace=True)

In [16]:
df

,open,high,low,close,volume
date,,,,,
2015-01-02 00:05:00,1.21031,1.21036,1.21026,1.21026,32
2015-01-02 00:10:00,1.21026,1.21068,1.21019,1.21028,116
2015-01-02 00:15:00,1.21029,1.21064,1.20989,1.21004,187
2015-01-02 00:20:00,1.21003,1.21030,1.20992,1.21017,67
2015-01-02 00:25:00,1.21023,1.21025,1.21013,1.21016,47
...,...,...,...,...,...
2023-11-17 22:35:00,1.09032,1.09043,1.09025,1.09038,259
2023-11-17 22:40:00,1.09038,1.09046,1.09028,1.09039,237
2023-11-17 22:45:00,1.09038,1.09053,1.09034,1.09048,155


In [19]:
df.isna().mean()

open      0.0
high      0.0
low       0.0
close     0.0
volume    0.0
dtype: float64

In [22]:
df.describe(include='all').drop('count')

,open,high,low,close,volume
mean,1.123354,1.123564,1.123145,1.123354,383.478273
std,0.053407,0.053390,0.053423,0.053407,411.046306
min,0.954010,0.954640,0.953590,0.953980,1.000000
25%,1.088330,1.088560,1.088100,1.088340,145.000000
50%,1.120990,1.121190,1.120800,1.120990,277.000000
75%,1.164190,1.164390,1.164000,1.164190,482.000000
max,1.255310,1.255510,1.254850,1.255310,15079.000000


# Функции предобработки

## Функция автоматического подбора номера сохраняемой модели

In [33]:
import os
import glob

def get_savemodel_name(name:str, mod:str|None=None, direct:str='Models\\') -> str:
    file_list = glob.glob(f'{direct}{name}_*.cbm')
    if len(file_list) == 0:
        next_file_name = f'{name}_1'
    else:
        latest_file = max(file_list, key=os.path.getctime)
        last_file_num = int(latest_file.split('_')[1][:-4])
        next_file_name = f'{name}_{last_file_num+1}'
    
    if mod is not None:
        next_file_name += f'_{mod}'
    
    next_file_name += '.cbm'
    
    print(next_file_name)
    return f'{direct}{next_file_name}'

## Кастомные индикаторы

In [43]:
def data_scale(df: pd.DataFrame, scale=1) -> pd.DataFrame:
    if scale == 1: return df
    df['open']  = df['open'].apply(lambda x:  x * scale)
    df['high']  = df['high'].apply(lambda x:   x * scale)
    df['low']   = df['low'].apply(lambda x:   x * scale)
    df['close'] = df['close'].apply(lambda x: x * scale)
    return df

In [46]:
def set_shadows(df: pd.DataFrame, segments=1) -> pd.DataFrame:
    df['grn_shd'] = df['high'] - df['close']
    df['red_shd'] = df['open'] - df['low']

    for i in range(segments):
        shift_ = i + 1
        df[f'grn_shd_{shift_}'] = df['high'].shift(shift_) - df['close'].shift(shift_)
        df[f'red_shd_{shift_}'] = df['open'].shift(shift_) - df['low'].shift(shift_)

    return df

In [49]:
def set_trend(df: pd.DataFrame, segments=0) -> pd.DataFrame:
    df['trend_cls'] = (df['close'] > df['open']) * 1
    if segments > 0:
        for i in range(segments):
            shift_ = i + 1
            df[f'trend_cls_{shift_}'] = df['trend_cls'].shift(shift_).fillna(0).astype(int)
    return df

In [52]:
def set_volumes(df: pd.DataFrame, segments=1) -> pd.DataFrame:
    for i in range(segments):
        shift_ = i + 1
        df[f'volume_{shift_}'] = df['volume'].shift(shift_).fillna(0).astype(int)
    return df

## Функции финансовых индикаторов

In [69]:
# Настройки
short_ma    = 5
long_ma     = 12
rsi_period  = 14
oversold    = 25
overbuy     = 75
k_period    = 14
d_period    = 3


def cls_over(row):
    if row > overbuy: return 1
    elif row < oversold: return -1
    else: return 0

In [72]:
# стохастический осциллятор
def set_stoch(df: pd.DataFrame, segments: int = 0, abs_seg: bool = True, clas:bool=True) -> pd.DataFrame:
    n_high = df['high'].rolling(k_period).max()
    n_low = df['low'].rolling(k_period).min()
    df['stoch'] = (df['close'] - n_low) / (n_high - n_low) * 100
    df['stoch'] = (df['close'] - n_low) / (n_high - n_low)
    # df['stoch_d'] = df['stoch'].rolling(d_period).mean()

    if clas:
        df['stoch_cls'] = pd.Series(df['stoch'] * 100).apply(cls_over)
    if segments > 0:
        for i in range(segments):
            shift_ = i+1
            if abs_seg:
                df[f'stoch_{shift_}'] = df['stoch'].shift(shift_)
            if clas:
                df[f'stoch_cls_{shift_}'] = df['stoch_cls'].shift(shift_)

    return df

In [75]:
def set_rsi(df:pd.DataFrame, segments:int=0, abs_seg:bool=True, clas:bool=True):

    up = pd.Series(np.maximum(df['close'].diff(), 0))
    down = pd.Series(np.maximum(-df['close'].diff(), 0))
    rs = up.rolling(rsi_period).mean() / down.rolling(rsi_period).mean()
    df['rsi'] = 100 - 100/(1 + rs)
    if clas:
        df['rsi_cls'] = df['rsi'].apply(cls_over)

    if segments > 0:
        for i in range(segments):
            shift_ = i + 1
            if abs_seg:
                df[f'rsi_{shift_}'] = df['rsi'].shift(shift_)
            if clas:
                df[f'rsi_cls_{shift_}'] = df['rsi_cls'].shift(shift_)

    del up
    del down
    del rs

    return df

In [81]:
def set_sma(df:pd.DataFrame, segments:int=0):
    df['sma'] = df['close'].rolling(short_ma).mean()
    if segments > 0:
        for i in range(segments):
            shift_ = i + 1
            df[f'sma_{shift_}'] = df['sma'].shift(shift_)

    return df

In [84]:
def set_lma(df:pd.DataFrame, segments:int=0):
    df['lma'] = df['close'].rolling(long_ma).mean()
    if segments > 0:
        for i in range(segments):
            shift_ = i + 1
            df[f'lma_{shift_}'] = df['lma'].shift(shift_)

    return df